# Lectura 45: Listas y arreglos

Polars permite trabajar con columnas de tipo Lista: es decir, columnas donde cada fila es una lista de elementos homogéneos, de diferentes longitudes. Polars también tiene un tipo de datos Array, que es análogo a los objetos ndarray de NumPy, donde la longitud es idéntica en todas las filas.

Debemos tener en cuenta que esto es diferente a una lista de Python, donde los elementos pueden ser de cualquier tipo.

## Listas

Para esta parte de la lección estaremos trabajando con un dataset que contiene el promedio diario de temperatura obtenido en las mayores ciudades del mundo. El cual puede ser obtenido desde Kaggle [en este enlace](https://www.kaggle.com/datasets/sudalairajkumar/daily-temperature-of-major-cities).

In [1]:
import polars as pl

city_temperatures = (
    pl.scan_csv('./data/city_temperature.csv')
)

city_temperatures.collect()

Region,Country,State,City,Month,Day,Year,AvgTemperature
str,str,str,str,i64,i64,i64,f64
"""Africa""","""Algeria""",null,"""Algiers""",1,1,1995,64.2
"""Africa""","""Algeria""",null,"""Algiers""",1,2,1995,49.4
"""Africa""","""Algeria""",null,"""Algiers""",1,3,1995,48.8
"""Africa""","""Algeria""",null,"""Algiers""",1,4,1995,46.4
"""Africa""","""Algeria""",null,"""Algiers""",1,5,1995,47.9
…,…,…,…,…,…,…,…
"""North America""","""US""","""Additional Ter…","""San Juan Puert…",7,27,2013,82.4
"""North America""","""US""","""Additional Ter…","""San Juan Puert…",7,28,2013,81.6
"""North America""","""US""","""Additional Ter…","""San Juan Puert…",7,29,2013,84.2


Lo que vamos a realizar a continuación es generar una lista que contenga todos los promedios de temperaturas registrados en cada una de las ciudades en una lista para que así podamos ver como funciona la manipulación de listas en Polars.

In [4]:
avg_temp_city = (
    city_temperatures.group_by('City').agg(
    pl.col('AvgTemperature').implode().flatten()
    )
).collect()

avg_temp_city

City,AvgTemperature
str,list[f64]
"""Capetown""","[66.8, 67.8, … 54.0]"
"""Dakar""","[70.8, 71.7, … 70.7]"
"""Windhoek""","[74.7, 80.8, … 58.0]"
"""Cincinnati""","[41.1, 22.2, … 50.0]"
"""Islamabad""","[46.2, 49.2, … 77.5]"
…,…
"""Helsinki""","[34.9, 28.9, … 29.3]"
"""Portland""","[32.2, 31.5, … 57.5]"
"""Toronto""","[33.0, 25.2, … 68.7]"


## Operaciones en columnas de tipo `List`

Polars proporciona varias operaciones estándar en las columnas de tipo `List`. Si queremos las tres primeras temperaturas, podemos hacer un `head(3)`. Las últimas tres se pueden obtener mediante un `tail(3)` o, alternativamente, podemos usar `slice` donde se admite indexación negativa. También podemos identificar el número de observaciones mediante la función `len`.

In [5]:
from polars import col

avg_temp_city.select(
    col('City'),
    col('AvgTemperature'),
    col('AvgTemperature').list.head(3).alias('top_3'),
    col('AvgTemperature').list.tail(3).alias('last_3'),
    col('AvgTemperature').list.slice(-3).alias('slice'),
    col('AvgTemperature').list.len().alias('total')
)

City,AvgTemperature,top_3,last_3,slice,total
str,list[f64],list[f64],list[f64],list[f64],u32
"""Capetown""","[66.8, 67.8, … 54.0]","[66.8, 67.8, 66.9]","[64.0, 61.3, 54.0]","[64.0, 61.3, 54.0]",9266
"""Dakar""","[70.8, 71.7, … 70.7]","[70.8, 71.7, 71.7]","[72.9, 72.8, 70.7]","[72.9, 72.8, 70.7]",9266
"""Windhoek""","[74.7, 80.8, … 58.0]","[74.7, 80.8, 78.1]","[62.1, 63.1, 58.0]","[62.1, 63.1, 58.0]",9266
"""Cincinnati""","[41.1, 22.2, … 50.0]","[41.1, 22.2, 22.8]","[44.3, 47.8, 50.0]","[44.3, 47.8, 50.0]",9265
"""Islamabad""","[46.2, 49.2, … 77.5]","[46.2, 49.2, 48.8]","[83.5, 76.2, 77.5]","[83.5, 76.2, 77.5]",9265
…,…,…,…,…,…
"""Helsinki""","[34.9, 28.9, … 29.3]","[34.9, 28.9, 16.5]","[39.1, 40.0, 29.3]","[39.1, 40.0, 29.3]",9266
"""Portland""","[32.2, 31.5, … 57.5]","[32.2, 31.5, 23.9]","[68.3, 56.8, 57.5]","[68.3, 56.8, 57.5]",18530
"""Toronto""","[33.0, 25.2, … 68.7]","[33.0, 25.2, 21.5]","[64.8, 66.0, 68.7]","[64.8, 66.0, 68.7]",7101


Una vez que tenemos las temperaturas en una lista es muy fácil obtener por ejemplo el promedio mínimo y máximo de temperatura por ciudad, así como la temperatura media. Además, podríamos ver las ciudades con el mayor promedio de temperatura.

In [6]:
avg_temp_city.select(
    col('City'),
    col('AvgTemperature').list.min().alias('temp_min'),
    col('AvgTemperature').list.max().alias('temp_max'),
    col('AvgTemperature').list.mean().alias('temp_avg')
).sort('temp_avg', descending=True)

City,temp_min,temp_max,temp_avg
str,f64,f64,f64
"""Dubai""",-99.0,107.5,82.972631
"""Chennai (Madra…",-99.0,97.9,82.847021
"""Doha""",-99.0,108.5,82.235625
"""Abu Dhabi""",-99.0,107.3,82.192499
"""Niamey""",-99.0,102.8,81.951619
…,…,…,…
"""Frankfurt""",-99.0,85.2,-13.668786
"""Lilongwe""",-99.0,90.7,-20.585544
"""Georgetown""",-99.0,90.6,-22.10152


## Array

Los Array son un nuevo tipo de datos que se introdujo recientemente en Polars, (esto teniendo en cuenta la fecha en que se crea esta lección) y aún son bastante incipientes en cuanto a las características que ofrece. La principal diferencia entre una Lista y un Array es que este último se limita a tener el mismo número de elementos por fila, mientras que una Lista puede tener un número variable de elementos. Ambos aún requieren que el tipo de datos de cada elemento sea el mismo.

Vamos a agregar una nueva columna al DataFrame `avg_temp_city` que sea de tipo Array. Para ello necesitamos tener la misma cantidad de temperaturas por cada ciudad para crear una nueva columna de tipo Array. A partir de la función `gather` que está disponible en una columna de tipo lista vamos a tomar un subconjunto de solo tres temperaturas por cada ciudad y luego realizaremos un `cast` para obtener así la columna de tipo Array.

In [7]:
avg_temp_city_array = avg_temp_city.with_columns(
    temp_array=col('AvgTemperature').list.gather([0,1,2]).cast(pl.Array(inner=pl.Float64, width=3))
)

avg_temp_city_array

City,AvgTemperature,temp_array
str,list[f64],"array[f64, 3]"
"""Capetown""","[66.8, 67.8, … 54.0]","[66.8, 67.8, 66.9]"
"""Dakar""","[70.8, 71.7, … 70.7]","[70.8, 71.7, 71.7]"
"""Windhoek""","[74.7, 80.8, … 58.0]","[74.7, 80.8, 78.1]"
"""Cincinnati""","[41.1, 22.2, … 50.0]","[41.1, 22.2, 22.8]"
"""Islamabad""","[46.2, 49.2, … 77.5]","[46.2, 49.2, 48.8]"
…,…,…
"""Helsinki""","[34.9, 28.9, … 29.3]","[34.9, 28.9, 16.5]"
"""Portland""","[32.2, 31.5, … 57.5]","[32.2, 31.5, 23.9]"
"""Toronto""","[33.0, 25.2, … 68.7]","[33.0, 25.2, 21.5]"


Podemos acceder a los métodos de las columna de tipo Array a través de `.arr`. Veamos algunos ejemplos.

In [11]:
avg_temp_city_array.select(
    col('temp_array'),
    col('temp_array').arr.min().alias('min_sample'),
    col('temp_array').arr.max().alias('max_sample'),
    col('temp_array').arr.count_matches(33.0).alias('count_temp_33'),
    col('temp_array').arr.sort().alias('temp_sorted')
)

temp_array,min_sample,max_sample,count_temp_33,temp_sorted
"array[f64, 3]",f64,f64,u32,"array[f64, 3]"
"[66.8, 67.8, 66.9]",66.8,67.8,0,"[66.8, 66.9, 67.8]"
"[70.8, 71.7, 71.7]",70.8,71.7,0,"[70.8, 71.7, 71.7]"
"[74.7, 80.8, 78.1]",74.7,80.8,0,"[74.7, 78.1, 80.8]"
"[41.1, 22.2, 22.8]",22.2,41.1,0,"[22.2, 22.8, 41.1]"
"[46.2, 49.2, 48.8]",46.2,49.2,0,"[46.2, 48.8, 49.2]"
…,…,…,…,…
"[34.9, 28.9, 16.5]",16.5,34.9,0,"[16.5, 28.9, 34.9]"
"[32.2, 31.5, 23.9]",23.9,32.2,0,"[23.9, 31.5, 32.2]"
"[33.0, 25.2, 21.5]",21.5,33.0,1,"[21.5, 25.2, 33.0]"
